In [ ]:
!mkdir -p ~/.cloudvolume/secrets
!cp /notebooks/*.json ~/.cloudvolume/secrets/
!pip install cloud-volume
!pip install scikit-image

In [47]:
from cloudvolume import CloudVolume
from cloudvolume.lib import Bbox, Vec
import numpy as np
import torch

In [43]:
src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=2, 
                                                                parallel=2, fill_missing=True)
info = src.info
info['data_type'] = 'float32'
info['num_channels'] = 2
dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/vector_test", mip=2, 
                                info=info, fill_missing=True, bounded=False, non_aligned_writes=True)
dst.commit_info();

In [3]:
offset = Vec(102716, 107077, 526)
size = Vec(2048, 2048, 1)
full_bbox = Bbox(offset, offset+size)
src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_bbox))

In [4]:
def make_field(dim, k=-1):
    field = torch.zeros((1,dim,dim,2))
    for i in range(dim):
        for j in range(dim):
            if i < dim - j:
                field[0,i,j,0] = 1*k
                field[0,i,j,1] = 1*k
            else:
                field[0,i,j,0] = -1*k
                field[0,i,j,1] = -1*k
    return field

In [16]:
field = make_field(int(src_bbox.size3()[0]))

In [25]:
s = src_bbox.to_slices()

In [27]:
s + (slice(0,2),)

(slice(25679, 26191, None),
 slice(26769, 27281, None),
 slice(526, 527, None),
 slice(0, 2, None))

In [33]:
vectors = field.permute(1,2,0,3).numpy()

In [ ]:
def save_field(cv, bbox, field):
    s = bbox.to_slices() + (slice(0,2),)
    cv[s] = field.permute(1,2,0,3).numpy()    

In [45]:
dst[s + (slice(0,2),)] = field.permute(1,2,0,3).numpy()

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  5.60it/s]


Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  5.71it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  3.55it/s]



In [46]:
vectors

array([[[[-1., -1.]],

        [[-1., -1.]],

        [[-1., -1.]],

        ...,

        [[-1., -1.]],

        [[-1., -1.]],

        [[-1., -1.]]],


       [[[-1., -1.]],

        [[-1., -1.]],

        [[-1., -1.]],

        ...,

        [[-1., -1.]],

        [[-1., -1.]],

        [[ 1.,  1.]]],


       [[[-1., -1.]],

        [[-1., -1.]],

        [[-1., -1.]],

        ...,

        [[-1., -1.]],

        [[ 1.,  1.]],

        [[ 1.,  1.]]],


       ...,


       [[[-1., -1.]],

        [[-1., -1.]],

        [[-1., -1.]],

        ...,

        [[ 1.,  1.]],

        [[ 1.,  1.]],

        [[ 1.,  1.]]],


       [[[-1., -1.]],

        [[-1., -1.]],

        [[ 1.,  1.]],

        ...,

        [[ 1.,  1.]],

        [[ 1.,  1.]],

        [[ 1.,  1.]]],


       [[[-1., -1.]],

        [[ 1.,  1.]],

        [[ 1.,  1.]],

        ...,

        [[ 1.,  1.]],

        [[ 1.,  1.]],

        [[ 1.,  1.]]]], dtype=float32)